## Utils and definitions

In [1]:
import os
import pickle

import numpy as np
import pandas as pd

import torch
import torch.nn.functional as F
import torch_geometric.nn as pyg_nn
import torch_geometric.utils as pyg_utils
import torch.optim as optim


from datetime import datetime
from torch import nn
from torch_geometric.data import Data, Dataset, DataLoader
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops
from torch_scatter import scatter
from torch.utils.tensorboard import SummaryWriter

In [2]:
from dataloader import lmdb_dataset

In [3]:
def my_reshape(tensor):
    return torch.reshape(tensor, (tensor.shape[0], 1))

In [4]:
#чтобы тензор по умолчанию заводился на куде
if torch.cuda.is_available():
    torch.set_default_tensor_type('torch.cuda.FloatTensor')
    print('cuda')

cuda


In [5]:
#set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  
print(device)

cuda


In [6]:
#делаем из данных матрицу векторов-атомов, список рёбер (edge_index) и матрицу векторов-рёбер
def simple_preprocessing(batch):
    #spherical_radii = torch.Tensor(batch['spherical_domain_radii'])
    #spherical_radii = my_reshape(spherical_radii)
    
    tags = batch['tags'].long().to(device)
    tags = F.one_hot(tags, num_classes=3)
    
    atom_numbers = batch['atomic_numbers'].long().to(device)
    atom_numbers = F.one_hot(atom_numbers, num_classes=100)
    
    voronoi_volumes = batch['voronoi_volumes'].float().to(device)
    voronoi_volumes = my_reshape(voronoi_volumes)
    
    atom_features = (tags, atom_numbers, voronoi_volumes)#, spherical_radii)
    atom_embeds = torch.cat(atom_features, 1)
    
    edge_index = batch['edge_index_new'].long().to(device)
    
    distances = batch['distances_new'].float().to(device)
    distances = my_reshape(distances)
    
    angles = batch['contact_solid_angles'].float().to(device)
    angles = my_reshape(angles)
    
    edges_embeds = torch.cat((distances, angles), 1)
    
    
    return Data(x=atom_embeds.to(device), edge_index=edge_index.to(device), edge_attr=edges_embeds.to(device))

In [7]:
#датасет, который умеет возвращать эелемент и собственную длину
class Dataset(Dataset):

    def __init__(self, data, features_fields, target_field, type_='train', preprocessing=simple_preprocessing):
        
        self.data = lmdb_dataset({"src": data})
        self.length = len(self.data)
        #self.target = data[target_field]
        self.type_ = type_
        self.preprocessing = preprocessing
        self.target = target_field

    def __len__(self):
        return self.length

    def __getitem__(self, index):
        
        system = self.preprocessing(self.data[index])
        
        if self.type_ == 'train':
            y = self.data[index][self.target]
            
            return system, y

In [8]:
batch_size = 50
num_workers = 0

features_cols = ['atomic_numbers', 'edge_index_new', 'distances_new', 
                 'contact_solid_angles', 'tags', 'voronoi_volumes', 'spherical_domain_radii']

target_col = 'y_relaxed'

In [9]:
#инициализируем тренировочный датасети и тренировочный итератор
train_dataset_file_path= os.path.expanduser("../../ocp_datasets/data/is2re/10k/train/data_mod.lmdb")

training_set = Dataset(train_dataset_file_path, features_cols, target_col)
training_generator = DataLoader(training_set, batch_size=batch_size, num_workers=num_workers)

In [10]:
#инициализируем валидационный датасет и валидационный итератор
val_dataset_file_path = os.path.expanduser("../../ocp_datasets/data/is2re/all/val_ood_cat/data_mod.lmdb")

valid_set = Dataset(val_dataset_file_path, features_cols, target_col)
valid_generator = DataLoader(valid_set, batch_size=batch_size, num_workers=num_workers)

$$
\mathbf{x}_i^{(k)} = \gamma^{(k)} \left( \mathbf{x}_i^{(k-1)}, \square_{j \in \mathcal{N}(i)} \, \phi^{(k)}\left(\mathbf{x}_i^{(k-1)}, \mathbf{x}_j^{(k-1)},\mathbf{e}_{j,i}\right) \right)
$$

Гамма лежит в апдейт, квадратик в aggr, а фи в месседж; в этом примере гамма и фи -- умножение на матрицу после конкатенации, а квадратик -- суммирование

In [11]:
class GConv(MessagePassing):
    def __init__(self, dim_atom=103, dim_edge=2, out_channels=2):
        super(GConv, self).__init__(aggr='add')  # "Add" aggregation
        self.phi_output = 3
        self.lin_phi = torch.nn.Linear(dim_atom*2+dim_edge, self.phi_output, bias=False)
        self.lin_gamma = torch.nn.Linear(dim_atom + self.phi_output, out_channels, bias=False)
        self.nonlin = nn.Sigmoid()

    def forward(self, batch):
        x = batch['x']
        edge_index = batch['edge_index']
        edge_attr = batch['edge_attr']
        
        # x has shape [N -- количество атомов в системе(батче), in_channels -- размерность вектора-атома]
        # edge_index has shape [2, E] -- каждое ребро задаётся парой вершин

        # Start propagating messages. 
    
        return self.propagate(edge_index, x=x, edge_attr=edge_attr, size=None)  #не совсем понял что такое сайз

    def message(self, x, x_i, x_j, edge_attr):
        concatenated = torch.cat((x_i, x_j, edge_attr), 1)
        phi = self.lin_phi(concatenated)
        phi = self.nonlin(phi)
        return phi
        
    def update(self, aggr_out, x, edge_attr, edge_index):
                
        concatenated = torch.cat((x, aggr_out), 1)
        gamma = self.lin_gamma(concatenated)
        gamma = self.nonlin(gamma)

        return Data(x=gamma, edge_attr=edge_attr, edge_index=edge_index)

In [12]:
class ConvNN(nn.Module):
    
    def __init__(self, dim_atom=103, dim_edge=2):
        
        super().__init__()          
#         self.conv_1 = GConv(dim_atom=dim_atom, dim_edge=dim_edge, out_channels=dim_atom)
#         self.conv_2 = GConv(dim_atom=dim_atom, dim_edge=dim_edge, out_channels=dim_atom)
#         self.conv_3 = GConv(dim_atom=dim_atom, dim_edge=dim_edge, out_channels=dim_atom)
#         self.conv_4 = GConv(dim_atom=dim_atom, dim_edge=dim_edge, out_channels=dim_atom)
#         self.conv_5 = GConv(dim_atom=dim_atom, dim_edge=dim_edge, out_channels=dim_atom)
        self.conv_last = GConv(dim_atom=dim_atom, dim_edge=dim_edge, out_channels=2)
        
        self.lin = torch.nn.Linear(2, 1, bias=True)
        
    def forward(self, batch):
#         convoluted_1 = self.conv_1(batch)
#         convoluted_2 = self.conv_2(convoluted_1)
#         convoluted_3 = self.conv_3(convoluted_2)
#         convoluted_4 = self.conv_4(convoluted_3)
#         convoluted_5 = self.conv_5(convoluted_4)
        convoluted_last = self.conv_last(batch)['x']
        scattered = scatter(convoluted_last, batch['batch'], dim=0, reduce='sum')
        summed = scattered
        energy = self.lin(summed)
        
        return energy

In [13]:
def send_scalars(lr, loss, writer, step=-1, epoch=-1, type_='train'):
    if type_ == 'train':
        writer.add_scalar('lr per step on train', lr, step) 
        writer.add_scalar('loss per step on train', loss, step)
    if type_ == 'val':
        writer.add_scalar('loss per epoch on val', loss, epoch)

In [14]:
def send_hist(model, writer, step):
    for name, weight in model.named_parameters():
        writer.add_histogram(name, weight, step)

In [15]:
def train(model, iterator, optimizer, criterion, print_every=10, epoch=0, writer=None):
    
    epoch_loss = 0
    
    model.train()

    for i, (systems, ys) in enumerate(iterator):
        
        optimizer.zero_grad()
        predictions = model(systems).squeeze()
        
        loss = criterion(predictions.float(), ys.to(device).float())
        loss.backward()     
        
        optimizer.step()
        
        batch_loss = loss.item() 
        epoch_loss += batch_loss  
        
        if writer != None:
            
            lr = optimizer.param_groups[0]['lr']
            
            step = i + epoch*len(iterator)
            
            send_hist(model, writer, i)
            send_scalars(lr, batch_loss, writer, step=step, epoch=epoch, type_='train')
        
        if not (i+1) % print_every:
            print(f'step {i} from {len(iterator)} at epoch {epoch}')
            print(f'Loss: {batch_loss}')
        
    return epoch_loss / len(iterator)

In [16]:
def evaluate(model, iterator, criterion, epoch=0, writer=False):
    
    epoch_loss = 0
    
#    model.train(False)
    model.eval()  
    
    with torch.no_grad():
        for systems, ys in iterator:   

            predictions = model(systems).squeeze()
            loss = criterion(predictions.float(), ys.to(device).float())        

            epoch_loss += loss.item()
            
    overall_loss = epoch_loss / len(iterator)

    if writer != None:
        send_scalars(None, overall_loss, writer, step=None, epoch=epoch, type_='val')
                
    print(f'epoch loss {overall_loss}')
            
    return overall_loss

In [17]:
def inferens(model, iterator):
    y = torch.tensor([])

#    model.train(False)
    model.eval()  
    
    with torch.no_grad():
        for systems, ys in iterator:   
          predictions = model(systemhs).squeeze()
          y = torch.cat((y, predictions))
      
    return y

## DATA

In [18]:
df_train = []
df_val = []

## MODEL

In [19]:
#model
model = ConvNN(dim_atom=training_set[0][0].x.shape[1], dim_edge=training_set[0][0].edge_attr.shape[1])

#optimizer and loss
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.L1Loss()

#переносим на куду если она есть
model = model.to(device)
criterion = criterion.to(device)

In [20]:
timestamp = str(datetime.now().strftime("%Y-%m-%d-%H-%M-%S"))

print(timestamp)

2021-09-06-21-34-51


In [21]:
#tensorboard writer, при первом запуске надо руками сделать папку для логов

# server
#log_folder_path = "../../ocp_results/logs/tensorboard/out_base_model"

# colab
# log_folder_path = "/content/drive/MyDrive/ocp_results/logs/tensorboard/out_base_model"

# user_specific 
log_file_path = "../logs/tensorboard_airi"

writer = SummaryWriter(log_file_path + '/' + timestamp)

In [22]:
#граф модели
trace_system = dict(list(next(iter(training_generator))[0]))
writer.add_graph(model, trace_system)

## Training

In [23]:
%%time
loss = []
loss_eval = []
epochs = 10
print(timestamp)
print(f'Start training model {str(model)}')
for i in range(epochs):
    print(f'epoch {i}')
    loss.append(train(model, training_generator, optimizer, criterion, epoch=i, writer=writer))
    loss_eval.append(evaluate(model, valid_generator, criterion, epoch=i, writer=writer))

2021-09-06-21-34-51
Start training model ConvNN(
  (conv_last): GConv(
    (lin_phi): Linear(in_features=210, out_features=3, bias=False)
    (lin_gamma): Linear(in_features=107, out_features=2, bias=False)
    (nonlin): Sigmoid()
  )
  (lin): Linear(in_features=2, out_features=1, bias=True)
)
epoch 0
step 9 from 200 at epoch 0
Loss: 11.55709457397461
step 19 from 200 at epoch 0
Loss: 7.758169174194336
step 29 from 200 at epoch 0
Loss: 5.590822696685791
step 39 from 200 at epoch 0
Loss: 3.6836798191070557
step 49 from 200 at epoch 0
Loss: 4.06721305847168
step 59 from 200 at epoch 0
Loss: 3.8997697830200195
step 69 from 200 at epoch 0
Loss: 3.7400050163269043
step 79 from 200 at epoch 0
Loss: 2.746499538421631
step 89 from 200 at epoch 0
Loss: 2.751690626144409
step 99 from 200 at epoch 0
Loss: 2.7941832542419434
step 109 from 200 at epoch 0
Loss: 2.7602365016937256
step 119 from 200 at epoch 0
Loss: 2.8841552734375
step 129 from 200 at epoch 0
Loss: 2.1670889854431152
step 139 from 20

KeyboardInterrupt: 